# Chapter 11: Ray's Ecosystem and Beyond


You can run this notebook directly in
[Colab](https://colab.research.google.com/github/maxpumperla/learning_ray/blob/main/notebooks/ch_11_ecosystem.ipynb).

The book has been written for Ray 2.2.0,which at the time of writing has not
officially been released yet. If you are reading this and this version is already
available, you can install it using `pip install ray==2.2.0`. If not, you can
use a nightly wheel (here for Python 3.7 on Linux):

In [ ]:
! pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-3.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl

Should you not run this notebook in Colab and need another type of wheel, please
refer to Ray's [installation instructions for nightlies](https://docs.ray.io/en/latest/ray-overview/installation.html#install-nightlies).

For this chapter you will also need to install the following dependencies:

In [ ]:
! pip install "ray[air]"==2.2.0
! pip install requests torch torchvision mlflow gradio


To import utility files for this chapter, on Colab you will also have to clone
the repo and copy the code files to the base path of the runtime:

In [ ]:
!git clone https://github.com/maxpumperla/learning_ray
%cp -r learning_ray/notebooks/* .

In [ ]:
# tag::load_cifar[]
from torchvision import transforms, datasets


def load_cifar(train: bool):
    transform = transforms.Compose([  # <1>
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    return datasets.CIFAR10(  # <2>
        root="./data",
        download=True,
        train=train,  # <3>
        transform=transform
    )
# end::load_cifar[]

In [ ]:
# tag::std[]
from ray.data import from_torch


train_dataset = from_torch(load_cifar(train=True))
test_dataset = from_torch(load_cifar(train=False))
# end::std[]

In [ ]:
# tag::batch_conversion[]
import numpy as np


def to_labeled_image(batch):  # <1>
    return {
        "image": np.array([image.numpy() for image, _ in batch]),
        "label": np.array([label for _, label in batch]),
    }


train_dataset = train_dataset.map_batches(to_labeled_image)  # <2>
test_dataset = test_dataset.map_batches(to_labeled_image)
# end::batch_conversion[]

In [ ]:
# tag::torch_model[]
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
# end::torch_model[]

In [ ]:
# tag::torch_training_loop[]
from ray import train
from ray.air import session, Checkpoint


def train_loop(config):
    model = train.torch.prepare_model(Net())  # <1>
    loss_fct = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    train_batches = session.get_dataset_shard("train").iter_torch_batches(  # <2>
        batch_size=config["batch_size"],
    )

    for epoch in range(config["epochs"]):
        running_loss = 0.0
        for i, data in enumerate(train_batches):
            inputs, labels = data["image"], data["label"]  # <3>

            optimizer.zero_grad()  # <4>
            forward_outputs = model(inputs)
            loss = loss_fct(forward_outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()  # <5>
            if i % 1000 == 0:
                print(f"[{epoch + 1}, {i + 1:4d}] loss: "
                      f"{running_loss / 1000:.3f}")
                running_loss = 0.0

        session.report(  # <6>
            dict(running_loss=running_loss),
            checkpoint=Checkpoint.from_dict(
                dict(model=model.module.state_dict())
            ),
        )
# end::torch_training_loop[]

In [ ]:
# tag::torch_trainer[]
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig, RunConfig
from ray.air.callbacks.mlflow import MLflowLoggerCallback


trainer = TorchTrainer(
    train_loop_per_worker=train_loop,
    train_loop_config={"batch_size": 10, "epochs": 5},
    datasets={"train": train_dataset},
    scaling_config=ScalingConfig(num_workers=2),
    run_config=RunConfig(callbacks=[
        MLflowLoggerCallback(experiment_name="torch_trainer")
    ])

)
result = trainer.fit()
# end::torch_trainer[]

In [ ]:
# tag::store_checkpoint[]
CHECKPOINT_PATH = "torch_checkpoint"
result.checkpoint.to_directory(CHECKPOINT_PATH)
# end::store_checkpoint[]

import sys
sys.exit("End of executable script. Stopping smoke tests here.")

In [ ]:
# tag::custom_data[]
from ray.data import read_datasource, datasource


class SnowflakeDatasource(datasource.Datasource):
    pass


dataset = read_datasource(SnowflakeDatasource(), ...)
# end::custom_data[]

In [ ]:
# tag::custom_trainer[]
from ray.train.data_parallel_trainer import DataParallelTrainer


class JaxTrainer(DataParallelTrainer):
    pass


trainer = JaxTrainer(
    ...,
    scaling_config=ScalingConfig(...),
    datasets=dict(train=dataset),
)
# end::custom_trainer[]

In [ ]:
# tag::custom_tuner[]
from ray.tune import logger, tuner
from ray.air.config import RunConfig


class NeptuneCallback(logger.LoggerCallback):
    pass


tuner = tuner.Tuner(
    trainer,
    run_config=RunConfig(callbacks=[NeptuneCallback()])
)
# end::custom_tuner[]